In [79]:
import pandas as pd
from  datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, monotonically_increasing_id, udf
from pyspark.sql.types import (StructType,
                               StructField,
                               StringType,
                               IntegerType,
                               DoubleType,
                               DateType,
                               FloatType,
                               LongType,
                               TimestampType)

In [2]:
spark = SparkSession.builder.\
    appName("data_spark_on_emr").\
    config("spark.jars.packages","saurfang:spark-sas7bdat:3.0.0-s_2.12").\
    getOrCreate()

22/05/27 06:37:59 WARN Utils: Your hostname, OneForAll-NickdeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.101 instead (on interface en0)
22/05/27 06:37:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/Cellar/apache-spark/3.2.1/libexec/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/oneforall_nick/.ivy2/cache
The jars for the packages stored in: /Users/oneforall_nick/.ivy2/jars
saurfang#spark-sas7bdat added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0b15e881-3f97-41b5-a5aa-526a6701d837;1.0
	confs: [default]
	found saurfang#spark-sas7bdat;3.0.0-s_2.12 in spark-packages
	found com.epam#parso;2.0.11 in central
	found org.slf4j#slf4j-api;1.7.5 in central
	found org.apache.logging.log4j#log4j-api-scala_2.12;12.0 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.apache.logging.log4j#log4j-api;2.13.2 in central
:: resolution report :: resolve 318ms :: artifacts dl 30ms
	:: modules in use:
	com.epam#parso;2.0.11 from central in [default]
	org.apache.logging.log4j#log4j-api;2.13.2 from central in [default]
	org.apache.logging.log4j#log4j-api-scala_2.12;12.0 from central in [default]
	org.scala-lang#scala-reflect;2.12.10 from central in [default]
	org.slf4j#slf4j-api;1.7.5 fro

In [3]:
spark

In [4]:
# Stop spark session if I don't need it.
# spark.stop()

In [5]:
# spark session setting configuration
spark.conf.set("spark.sql.shuffle.partitions", "5")
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1653604681497'),
 ('spark.sql.parquet.binaryAsString', 'true'),
 ('spark.driver.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.memory', '4g'),
 ('spark.driver.port', '49487'),
 ('spark.driver.host', '10.0.0.101'),
 ('spark.executor.id', 'driver'),
 ('spark.jars',
  'file:///Users/oneforall_nick/.ivy2/jars/saurfang_spark-sas7bdat-3.0.0-s_2.12.jar,file:///Users/oneforall_nick/.ivy2/jars/com.epam_parso-2.0.11.jar,file:///Users/oneforall_nick/.ivy2/jars/org.apache.logging.log4j_log4j-api-scala_2.12-12.0.jar,file:///Users/oneforall_nick/.ivy2/jars/org.slf4j_slf4j-api-1.7.5.jar,file:///Users/oneforall_nick/.ivy2/jars/org.scala-lang_scala-reflect-2.12.10.jar,file:///Users/oneforall_nick/.ivy2/jars/org.apache.logging.log4j_log4j-api-2.13.2.jar'),
 ('spark.sql.warehouse.dir',
  'file:/Users/oneforall_nick/workspace/Udacity_capstone_project/aws_emr_steps/spark-warehouse'),
 ('spark.app.initial.jar.

In [ ]:
# READ ALL DATA AND SAVE TO simulation data file, and then analysis those data in spark.
# ****** Read data from local file system(be a s3) to spark DataFrame ******
# All data format will be saved as a parquet file.

# file path: data >> immigration_data
# There are three tables: immigration_table, immigration_personal_table and immigration_label_table
# ***immigration_table, immigration_personal_table and immigration_label_table schema***
""""Table: immigration_main_information schema
pk: cicid -> imm_main_cic_id
1. i94yr  -> imm_year
2. i94mon -> imm_month
3. i94citi&i94res -> imm_citi_res -> imm_cntyl
4. i94visa -> imm_visa
    three categories:
        1 = Business
        2 = Pleasure
        3 = Student
5. i94port -> imm_port
6. arrdate -> imm_arrival_date:
7. i94mode -> imm_model:
    four categories:
        1 = 'Air'
	    2 = 'Sea'
	    3 = 'Land'
	    9 = 'Not reported'
8. i94addr -> imm_address
    ex: 'AL'='ALABAMA'
9. airline -> imm_airline
10 fltno -> imm_flight_no
    schema: StringType()
"""


"""Table: immigration_personal schema -> According to this person data that I will make a core data table to display notifications information.
pk: cicid -> imm_per_cic_id
    schema: StringType()
1. biryear -> imm_person_birth_year
    schema: IntegerType()
2. gender -> imm_person_gender
    schema: StringType()
3. visatype -> imm_person_visa_type
    schema: StringType()
"""

"""Table: immigration_label schema


"""


In [89]:
"""Table: immigration_label schema


"""

label_data = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/immigration_data/immigration_labels_descriptions.SAS"

# Get data column attributes
from sas7bdat import SAS7BDAT

# Using the library for knowing more data information
imm_label_data_read = SAS7BDAT(imm_data)

for i in imm_label_data_read

TypeError: __repr__ returned non-string (type bytes)

In [15]:
# file path: data >> news_data
"""Table: news_article schema
pk: cord_uid -> news_cord_uid
1. source_x -> news_source
    schema: StringType()
2. title -> news_title
    schema: StringType()
3. license -> news_licence
    schema: StringType()
4. abstract -> news_abstract
    schema: StringType()
5. publish_time -> news_publish_time (fk)
    schema: TimestampType()
6. authors -> news_authors
    schema: StringType()
7. url -> news_url
    schema: StringType()
"""
data_news = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/news_data/metadata.csv"

df_news = pd.read_csv(data_news, header=0, delimiter=',')
df_news.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389


In [12]:
# file path: data >> usCitiesDemographics_data
data_us_cities_demographics = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/usCitiesDemographics_data/usCitiesDemo.csv"

df_data_us_cities_demographics = pd.read_csv(data_us_cities_demographics, header=0, delimiter=";")

df_data_us_cities_demographics


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
...,...,...,...,...,...,...,...,...,...,...,...,...
2886,Stockton,California,32.5,150976.0,154674.0,305650,12822.0,79583.0,3.16,CA,American Indian and Alaska Native,19834
2887,Southfield,Michigan,41.6,31369.0,41808.0,73177,4035.0,4011.0,2.27,MI,American Indian and Alaska Native,983
2888,Indianapolis,Indiana,34.1,410615.0,437808.0,848423,42186.0,72456.0,2.53,IN,White,553665
2889,Somerville,Massachusetts,31.0,41028.0,39306.0,80334,2103.0,22292.0,2.43,MA,American Indian and Alaska Native,374


In [31]:
# Create a us-cities data dimension table
"""Table: us_cities_demographics schema
pk: generated -> cidemo_id
    schema: IntegerType()
1. City -> cidemo_city
    schema: StringType()
2. State -> cidemo_state
    schema: StringType()
3. Median Age -> cidemo_median_age
    schema: FloatType()
4. Total Population -> cidemo_total_population
    schema: IntegerType()
5. State Code -> cidemo_state_code (fk)
    schema: StringType()
6. Count -> cidemo_count
    schema: IntegerType()
"""

# TODO -> Must be defined a function that generated each table schema:
new_data_schema = StructType([\
    StructField(name="cidemo_city", dataType=StringType(), nullable=True),
    StructField(name="cidemo_state", dataType=StringType(), nullable=True),
    StructField(name="cidemo_median_age", dataType=FloatType(), nullable=True),
    StructField(name="cidemo_total_population", dataType=IntegerType(), nullable=True),
    StructField(name="cidemo_state_code", dataType=StringType(), nullable=True),
    StructField(name="cidemo_count", dataType=IntegerType(), nullable=True)
])

# Using pyspark to read csv file
pdf_news = spark.read.options(delimiter=';').csv(data_us_cities_demographics, schema=new_data_schema)
# pdf_news.printSchema()
"""
root
 |-- cidemo_city: string (nullable = true)
 |-- cidemo_state: string (nullable = true)
 |-- cidemo_median_age: float (nullable = true)
 |-- cidemo_total_population: integer (nullable = true)
 |-- cidemo_state_code: string (nullable = true)
 |-- cidemo_count: integer (nullable = true)
"""
# Auto-generated series of id
pdf_news = pdf_news.withColumn("cidemo_id", monotonically_increasing_id())
pdf_news.show(n=5, truncate=10)

+-----------+------------+-----------------+-----------------------+-----------------+------------+---------+
|cidemo_city|cidemo_state|cidemo_median_age|cidemo_total_population|cidemo_state_code|cidemo_count|cidemo_id|
+-----------+------------+-----------------+-----------------------+-----------------+------------+---------+
|       City|       State|             null|                   null|       Female ...|        null|        0|
| Silver ...|    Maryland|             33.8|                  40601|            41862|       82463|        1|
|     Quincy|  Massach...|             41.0|                  44129|            49500|       93629|        2|
|     Hoover|     Alabama|             38.5|                  38040|            46799|       84839|        3|
| Rancho ...|  California|             34.5|                  88127|            87105|      175232|        4|
+-----------+------------+-----------------+-----------------------+-----------------+------------+---------+
only showi

In [15]:
# TODO: This code block is be placed in analysis jupyter notebook files
imm_data = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/immigration_data/immigration_apr16_sub.sas7bdat"


df_imm_data = spark.read.format("com.github.saurfang.sas.spark").load(imm_data)


In [19]:
type(df_imm_data)


pyspark.sql.dataframe.DataFrame

In [37]:
"""Table: immigration_personal schema -> According to this person data that I will make a core data table to display notifications information.
pk: cicid -> imm_per_cic_id
    schema: StringType()
1. biryear -> imm_person_birth_year
    schema: IntegerType()
2. gender -> imm_person_gender
    schema: StringType()
3. visatype -> imm_person_visa_type
    schema: StringType()
"""
# show(n=5, truncate=5)
df_immigration_personal = df_imm_data.withColumn("imm_per_cic_id", col("cicid").cast("String"))\
           .withColumn("imm_person_birth_year", col("biryear").cast("Integer"))\
           .withColumn("imm_person_gender", col("gender").cast("String"))\
           .withColumn("imm_visatype", col("visatype").cast("String")).select(col("imm_per_cic_id"), \
                                                                              col("imm_person_birth_year"), \
                                                                              col("imm_person_gender"), \
                                                                              col("imm_visatype")).show(n=5, truncate=5)


+--------------+---------------------+-----------------+------------+
|imm_per_cic_id|imm_person_birth_year|imm_person_gender|imm_visatype|
+--------------+---------------------+-----------------+------------+
|           6.0|                 1979|             null|          B2|
|           7.0|                 1991|                M|          F1|
|          15.0|                 1961|                M|          B2|
|          16.0|                 1988|             null|          B2|
|          17.0|                 2012|             null|          B2|
+--------------+---------------------+-----------------+------------+
only showing top 5 rows



In [84]:
""""Table: immigration_main_information schema
pk: cicid -> imm_main_cic_id
1. i94yr: 4 digit year of the arrival  -> imm_year
2. i94mon: numeric month of the arrival -> imm_month
3. i94citi&i94res: 3 digit code of origin city -> imm_citi_res -> imm_cntyl
4. i94visa: reason for immigration -> imm_visa
    three categories:
        1 = Business
        2 = Pleasure
        3 = Student
5. i94port: 3 character code of destination city --> Foreign key (used to map to USDemographics and City Temperature data) -> imm_port
6. arrdate: arrival date of the departure -> imm_arrival_date:
7. deptdate: departure date
date_add
7. i94mode: 1 digit travel code -> imm_model:
    four categories:
        1 = 'Air'
	    2 = 'Sea'
	    3 = 'Land'
	    9 = 'Not reported'
8. i94addr -> imm_address
    ex: 'AL'='ALABAMA'
9. airline -> imm_airline
10 fltno -> imm_flight_no
    schema: StringType()
"""


def convert_to_datetime(days: DoubleType) -> datetime:
    """convert_to_datetime converts days to datetime format

    Args:
        days (DoubleType): from sas arrive or departure date

    Returns:
        datetime: added days to datetime format result.
    """
    if days is not None:
        date = datetime.strptime('1960-01-01', '%Y-%m-%d')

        return date + timedelta(days=days)

udf_convert_to_datetime = udf(lambda x: convert_to_datetime(x), DateType())

# df_imm_data.show(n=5, truncate=5)
# t1 = df_imm_data.withColumn("imm_depdate", col("depdate")).select(col("imm_depdate")).printSchema()

immigration_main_information = df_imm_data.withColumn("imm_main_cic_id", col("cicid").cast("Integer"))\
            .withColumn("imm_year", col("i94yr").cast("Integer"))\
                .withColumn("imm_month", col("i94mon").cast("Integer"))\
                    .withColumn("imm_cntyl", col("i94cit").cast("Integer"))\
                        .withColumn("imm_visa", col("i94visa").cast("Integer"))\
                            .withColumn("imm_port", col("i94port").cast("String"))\
                                .withColumn("imm_arrival_date", udf_convert_to_datetime(col("arrdate")))\
                                    .withColumn("imm_departure_date", udf_convert_to_datetime(col("depdate")))\
                                        .withColumn("imm_model", col("i94mode").cast("Integer"))\
                                            .withColumn("imm_address", col("i94addr").cast("String"))\
                                                .withColumn("imm_airline", col("airline").cast("String"))\
                                                    .withColumn("imm_flight_no", col("fltno").cast("String"))\
        .select(col('imm_main_cic_id'), \
                    col('imm_year'),\
                        col('imm_month'),\
                            col('imm_cntyl'),\
                                col('imm_visa'),\
                                    col('imm_port'),\
                                        col('imm_arrival_date'),\
                                            col('imm_departure_date'),\
                                                col('imm_model'),\
                                                    col('imm_address'),\
                                                        col('imm_airline'),\
                                                            col('imm_flight_no')).show(n=5, truncate=10)


root
 |-- imm_depdate: double (nullable = true)

+---------------+--------+---------+---------+--------+--------+----------------+------------------+---------+-----------+-----------+-------------+
|imm_main_cic_id|imm_year|imm_month|imm_cntyl|imm_visa|imm_port|imm_arrival_date|imm_departure_date|imm_model|imm_address|imm_airline|imm_flight_no|
+---------------+--------+---------+---------+--------+--------+----------------+------------------+---------+-----------+-----------+-------------+
|              6|    2016|        4|      692|       2|     XXX|      2016-04-29|              null|     null|       null|       null|         null|
|              7|    2016|        4|      254|       3|     ATL|      2016-04-07|              null|        1|         AL|       null|        00296|
|             15|    2016|        4|      101|       2|     WAS|      2016-04-01|        2016-08-25|        1|         MI|         OS|           93|
|             16|    2016|        4|      101|       2|  

In [14]:
# Get data column attributes
# from sas7bdat import SAS7BDAT

# Using the library for knowing more data information
# imm_data_read = SAS7BDAT(imm_data)

# imm_data_read.header
# for i in imm_data_read.columns:
#     print("col_id ", i.col_id)
#     print("  name",  i.name.decode(encoding ='utf-8'))
#     print("  label", i.label.decode(encoding ='utf-8'))
#     print("  format", i.format)
#     print("  type", i.type)
#     print("  length", i.length)

In [56]:
# Dealing with multiple data files
# TODO: Make a def for doing this.
# Method1: Using pandas to read file.
# pdf_immigration = pd.read_sas(imm_data, format='sas7bdat', iterator=True, chunksize=5000000)
# pdf_immigration = pd.read_sas(imm_data, format='sas7bdat')

# imm_chunks_1 = list(pdf_immigration)


In [50]:
# imm_chunks_1[['cicid', 'biryear', 'gender', 'visatype']]
# imm_chunks_1[:]
# imm_person = []
# for data in imm_chunks_1:
#     imm_person.append(data[['cicid', 'biryear', 'gender', 'visatype']])

In [10]:
from pandas.io.sas.sas7bdat import SAS7BDATReader

rdr = SAS7BDATReader(imm_data, convert_header_text=False)
df3 = rdr.read()

In [14]:
# rdd.take(7)
df3.head()

,b'cicid',b'i94yr',b'i94mon',b'i94cit',b'i94res',b'i94port',b'arrdate',b'i94mode',b'i94addr',b'depdate',...,b'entdepu',b'matflag',b'biryear',b'dtaddto',b'gender',b'insnum',b'airline',b'admnum',b'fltno',b'visatype'
0,6.0,2016.0,4.0,692.0,692.0,b'XXX',20573.0,NaN,NaN,NaN,...,b'U',NaN,1979.0,b'10282016',NaN,NaN,NaN,1.897628e+09,NaN,b'B2'
1,7.0,2016.0,4.0,254.0,276.0,b'ATL',20551.0,1.0,b'AL',NaN,...,b'Y',NaN,1991.0,b'D/S',b'M',NaN,NaN,3.736796e+09,b'00296',b'F1'
2,15.0,2016.0,4.0,101.0,101.0,b'WAS',20545.0,1.0,b'MI',20691.0,...,NaN,b'M',1961.0,b'09302016',b'M',NaN,b'OS',6.666432e+08,b'93',b'B2'
3,16.0,2016.0,4.0,101.0,101.0,b'NYC',20545.0,1.0,b'MA',20567.0,...,NaN,b'M',1988.0,b'09302016',NaN,NaN,b'AA',9.246846e+10,b'00199',b'B2'
4,17.0,2016.0,4.0,101.0,101.0,b'NYC',20545.0,1.0,b'MA',20567.0,...,NaN,b'M',2012.0,b'09302016',NaN,NaN,b'AA',9.246846e+10,b'00199',b'B2'
